In [14]:
from datasets import load_dataset, Dataset, Features, Value, ClassLabel

In [4]:
import pandas as pd

# Load from single JSON file
df = pd.read_json("data/m1_preference_data.json") 

In [8]:
df

,course_id,question_id,question_body,question_options,question_answer,question_type,preferences
0,15000,0,Consider the following contains function defin...,None,"In the same manner, $D(n) = D(n/2) + \Theta(n)...",open_answer,[{'question_body': 'Consider the following con...
1,15000,3,What is the asymptotic work of <code>parGroupy...,None,$\Theta(n)$,open_answer,[{'A': 'To answer your question about the asym...
2,15000,4,"We have a collection of rectangles in a plane,...",None,"case class Point(x: Int, y: Int) case class Re...",open_answer,[{'question_body': 'We have a collection of re...
3,15005,5,Which of the following scheduler policies are ...,"[FIFO (First In, First Out), SJF (Shortest Job...","['STCF (Shortest Time to Completion First)', '...",mcq,[{'question_body': 'Which of the following sch...
4,15005,6,Which of the following are correct implementat...,[c \n void acquire(struct lock *l)\n {\n ...,['c \n void acquire(struct lock *l)\n {\n ...,mcq,[{'question_body': 'Which of the following are...
...,...,...,...,...,...,...,...
1259,15003,7364,Implement a reliable broadcast algorithm witho...,None,Use a step of all-to-all communication.\nIn pa...,open_answer,[{'question': 'Implement a reliable broadcast ...
1260,15003,7365,Byzantine consistent broadcast (BCB) assumes o...,None,Let us assume that F processes in the system c...,open_answer,[{'question_body': 'Byzantine consistent broad...
1261,15003,7366,"If process i fails, then eventually all proces...",None,"True: Nothing can be said about process i, bec...",open_answer,"[{'question_body': 'If process i fails, then e..."
1262,15003,7368,What happens in the reliable broadcast algorit...,None,"Suppose that completeness is violated. Then, t...",open_answer,[{'question_body': 'What happens in the reliab...


In [9]:
df_filtered = df[["course_id", "question_id", "question_body", "question_answer", "question_type"]]

In [10]:
df_filtered

,course_id,question_id,question_body,question_answer,question_type
0,15000,0,Consider the following contains function defin...,"In the same manner, $D(n) = D(n/2) + \Theta(n)...",open_answer
1,15000,3,What is the asymptotic work of <code>parGroupy...,$\Theta(n)$,open_answer
2,15000,4,"We have a collection of rectangles in a plane,...","case class Point(x: Int, y: Int) case class Re...",open_answer
3,15005,5,Which of the following scheduler policies are ...,"['STCF (Shortest Time to Completion First)', '...",mcq
4,15005,6,Which of the following are correct implementat...,['c \n void acquire(struct lock *l)\n {\n ...,mcq
...,...,...,...,...,...
1259,15003,7364,Implement a reliable broadcast algorithm witho...,Use a step of all-to-all communication.\nIn pa...,open_answer
1260,15003,7365,Byzantine consistent broadcast (BCB) assumes o...,Let us assume that F processes in the system c...,open_answer
1261,15003,7366,"If process i fails, then eventually all proces...","True: Nothing can be said about process i, bec...",open_answer
1262,15003,7368,What happens in the reliable broadcast algorit...,"Suppose that completeness is violated. Then, t...",open_answer


In [18]:
print(df_filtered.dtypes)

course_id           int64
question_id         int64
question_body      object
question_answer    object
question_type      object
dtype: object


In [29]:
df_filtered.loc[:, "course_id"] = pd.to_numeric(df_filtered["course_id"], errors="coerce").astype("int32")
df_filtered.loc[:, "question_id"] = pd.to_numeric(df_filtered["question_id"], errors="coerce").astype("int32")
df_filtered.loc[:, "question_body"] = df_filtered["question_body"].astype("str")
df_filtered.loc[:, "question_answer"] = df_filtered["question_answer"].astype("str")

In [59]:
features = Features(
    {
        "course_id":  Value("int32"),
        "question_id": Value("int32"),
        "question_body": Value("string"),
        "question_answer": Value("string"),
        "question_type": Value("string")
    }
)


dataset = Dataset.from_pandas(df_filtered, preserve_index=False, features=features)
dataset.save_to_disk("data/mnlp_preference_data_question_answer")

Saving the dataset (0/1 shards):   0%|          | 0/1264 [00:00<?, ? examples/s]

In [50]:
df_chat = df_filtered.copy()
df_chat["messages"] = df_chat.apply(
    lambda row: [
        {"role": "user", "content": str(row["question_body"])},
        {"role": "assistant", "content": str(row["question_answer"])},
    ],
    axis=1,
)
df_chat["id"] = df_chat.apply(
    lambda row: f"epfl_mnlp_{row['course_id']}_{row['question_id']}", axis=1
)

df_chat["source"] = "epfl_mnlp"
df_chat = df_chat[["id", "messages", "source"]]

In [51]:
df_chat

,id,messages,source
0,epfl_mnlp_15000_0,"[{'role': 'user', 'content': 'Consider the fol...",epfl_mnlp
1,epfl_mnlp_15000_3,"[{'role': 'user', 'content': 'What is the asym...",epfl_mnlp
2,epfl_mnlp_15000_4,"[{'role': 'user', 'content': 'We have a collec...",epfl_mnlp
3,epfl_mnlp_15005_5,"[{'role': 'user', 'content': 'Which of the fol...",epfl_mnlp
4,epfl_mnlp_15005_6,"[{'role': 'user', 'content': 'Which of the fol...",epfl_mnlp
...,...,...,...
1259,epfl_mnlp_15003_7364,"[{'role': 'user', 'content': 'Implement a reli...",epfl_mnlp
1260,epfl_mnlp_15003_7365,"[{'role': 'user', 'content': 'Byzantine consis...",epfl_mnlp
1261,epfl_mnlp_15003_7366,"[{'role': 'user', 'content': 'If process i fai...",epfl_mnlp
1262,epfl_mnlp_15003_7368,"[{'role': 'user', 'content': 'What happens in ...",epfl_mnlp


In [62]:
dataset_chat = Dataset.from_pandas(df_chat, preserve_index=False)
dataset_chat.save_to_disk("data/mnlp_sft_data")

Saving the dataset (0/1 shards):   0%|          | 0/1264 [00:00<?, ? examples/s]